In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from cinnabar.plotting import _master_plot
from cinnabar.stats import bootstrap_statistic
import seaborn as sns
sns.set_theme()

In [ ]:
# load the basic edge data
edge_data_openfe = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/processed_results/combined_pymbar3_edge_data.csv")
fep_plus_edge_data = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/schrodinger_21_4_results/combined_schrodinger_ddg.csv")
fep_plus_dg_data = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/schrodinger_21_4_results/combined_schrodinger_dg.csv")
dg_data_openfe = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/processed_results/combined_pymbar3_calculated_dg_data.csv")
# load the fixed pfkfb3 dataset
pfkfb3_edge_data = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/processed_results/reruns/rerun_pymbar3_edge_data.csv")

In [ ]:
#load the private data
private_raw_edge_data = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/private_processed_results/combined_pymbar3_edge_data.csv")

In [ ]:
# manually drop pfkfb3 from the main df and add it from the rerun folder
# also drop edges marked as failed
complete_edges = edge_data_openfe[(edge_data_openfe["failed"] != True) & (edge_data_openfe["system name"] != "pfkfb3")].copy(deep=True).reset_index(drop=True)

all_openfe_edges = pd.concat([complete_edges, pfkfb3_edge_data])
# some ligands had missing exp data so drop differences involving those edges
all_openfe_edges = all_openfe_edges[all_openfe_edges["exp DDG (kcal/mol)"].notna()].copy(deep=True).reset_index(drop=True)
all_openfe_edges


In [ ]:
# load the dg data 
rerun_dg_data = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/processed_results/reruns/rerun_pymbar3_calculated_dg_data.csv")
# get the combined dg data
dg_data_openfe = dg_data_openfe[(dg_data_openfe["system name"] != "pfkfb3") & (dg_data_openfe["Exp DG (kcal/mol)"].notna())].copy(deep=True).reset_index(drop=True)
all_dg_data = pd.concat([dg_data_openfe, rerun_dg_data])
all_dg_data

In [ ]:
# add the FEP+ data to the all_dg_data df
fep_ccc, fep_plus_error = [], []
for _, row in all_dg_data.iterrows():
    # find the matching row in the fep_plus_dg_data
    matching_ds = fep_plus_dg_data[(fep_plus_dg_data["system name"] == row["system name"]) & (fep_plus_dg_data["system group"] == row["system group"])]
    matching_row = matching_ds[matching_ds["Ligand name"] == row["ligand name"]].iloc[0]
    fep_ccc.append(matching_row["Pred. dG (kcal/mol)"])
    fep_plus_error.append(matching_row["Pred. dG std. error (kcal/mol)"])
    # make sure that the exp data is close 
    assert abs(matching_row["Exp. dG (kcal/mol)"] - row["Exp DG (kcal/mol)"]) < 0.1, f"Mismatch in exp data for {row['system name']} {row['ligand name']}"

# add back to the df
all_dg_data["FEP+ DG (kcal/mol)"] = fep_ccc
all_dg_data["FEP+ DG std. error (kcal/mol)"] = fep_plus_error


In [ ]:
all_dg_data

In [ ]:
import numpy as np
# calculate the pairwise DDG for each system
dg_system_metrics = []  
for system in all_dg_data["system group"].unique():
    # get the edges for these systems
    system_df = all_dg_data[all_dg_data["system group"] == system].copy(deep=True).reset_index(drop=True)
    targets = system_df["system name"].unique()
    for target in targets:
        # get the edges for this target
        target_df = system_df[(system_df["system name"] == target)].copy(deep=True).reset_index(drop=True)
        target_data = {"System": target, "Class": system, "N_ligs": len(target_df)}
        ligands = target_df["ligand name"].unique()
        # calculate the bootsrap metrics
        for prediction, error, label in [("DG (kcal/mol)", "uncertainty (kcal/mol)", "OpenFE"), ("FEP+ DG (kcal/mol)", "FEP+ DG std. error (kcal/mol)", "FEP+")]:
            exp_ddgs, exp_error, prediction_ddgs, prediction_error = [], [], [], []
            for i, ligand_a in enumerate(ligands):
                for j, ligand_b in enumerate(ligands):
                    if i >= j:
                        continue
                    # get the exp ddg for this pair
                    ligand_a_exp = target_df[target_df["ligand name"] == ligand_a]["Exp DG (kcal/mol)"].values[0]
                    ligand_b_exp = target_df[target_df["ligand name"] == ligand_b]["Exp DG (kcal/mol)"].values[0]
                    exp_ddg = ligand_b_exp - ligand_a_exp
                    exp_ddgs.append(exp_ddg)
                    exp_error_a = target_df[target_df["ligand name"] == ligand_a]["Exp dDG (kcal/mol)"].values[0]
                    exp_error_b = target_df[target_df["ligand name"] == ligand_b]["Exp dDG (kcal/mol)"].values[0]
                    exp_error.append((exp_error_a ** 2 + exp_error_b ** 2) ** 0.5)
                    # get the prediction ddg for this pair
                    ligand_a_pred = target_df[target_df["ligand name"] == ligand_a][prediction].values[0]
                    ligand_b_pred = target_df[target_df["ligand name"] == ligand_b][prediction].values[0]
                    prediction_ddg = ligand_b_pred - ligand_a_pred
                    prediction_ddgs.append(prediction_ddg)
                    prediction_error_a = target_df[target_df["ligand name"] == ligand_a][error].values[0]
                    prediction_error_b = target_df[target_df["ligand name"] == ligand_b][error].values[0]
                    prediction_error.append((prediction_error_a ** 2 + prediction_error_b ** 2) ** 0.5)
            # convert to numpy arrays
            exp = np.array(exp_ddgs)
            exp_error = np.array(exp_error)
            prediction = np.array(prediction_ddgs)
            prediction_error = np.array(prediction_error)
            # manual bootstrap the ddg values
            nbootstrap = 1000
            rmse = np.zeros(nbootstrap)
            for i in range(nbootstrap):
                # sample with replacement
                indices = np.random.choice(len(exp), size=len(exp), replace=True)
                sampled_exp = exp[indices]
                sampled_prediction = prediction[indices]
                # calculate the RMSE
                rmse[i] = np.sqrt(np.mean((sampled_prediction - sampled_exp) ** 2))
            # calculate the RMSE mean and CI
            s = {
                "mle": np.mean(rmse),
                "low": np.percentile(rmse, 2.5),
                "high": np.percentile(rmse, 97.5)
            }
            target_data[f"{label} RMSE"] = s["mle"]
            target_data[f"{label} RMSE_lower"] = s["low"]
            target_data[f"{label} RMSE_upper"] = s["high"]
        # add the data to a new dataframe
        dg_system_metrics.append(target_data)
# create a new dataframe with all of the metrics calculated for each system
dg_system_metrics = pd.DataFrame(dg_system_metrics)
dg_system_metrics.sort_values(by=['Class', 'OpenFE RMSE'], inplace=True)
dg_system_metrics
        

In [ ]:
# how many systems have RMSE with openfe less that 1 kcal/mol compared to FEP+?
openfe_less_than_1 = dg_system_metrics[dg_system_metrics["OpenFE RMSE"] < 1].copy(deep=True).reset_index(drop=True)
fep_plus_less_than_1 = dg_system_metrics[dg_system_metrics["FEP+ RMSE"] < 1].copy(deep=True).reset_index(drop=True)
print(f"Number of systems with OpenFE RMSE < 1 kcal/mol: {len(openfe_less_than_1)}")
print(f"Number of systems with FEP+ RMSE < 1 kcal/mol: {len(fep_plus_less_than_1)}")

In [ ]:
# calculate the weighted RMSE for OpenFE and FEP+ with 95% CI from bootstrapping add to the dataframe
import numpy as np
nboots = 1000
openfe_rmse, fep_plus_rmse = np.zeros(nboots), np.zeros(nboots)
for i in range(nboots):
    # bootstrap the data
    bootstrapped_data = dg_system_metrics.sample(n=len(dg_system_metrics), replace=True)
    openfe_rmse[i] = np.sqrt(np.sum(bootstrapped_data["OpenFE RMSE"] ** 2 * bootstrapped_data["N_ligs"]) / np.sum(bootstrapped_data["N_ligs"]))
    fep_plus_rmse[i] = np.sqrt(np.sum(bootstrapped_data["FEP+ RMSE"] ** 2 * bootstrapped_data["N_ligs"]) / np.sum(bootstrapped_data["N_ligs"]))
    

row_data = {
    "System": "Weighted\nRMSE",
    "Class": "Overall",
    "N_ligs": len(dg_system_metrics),
    "OpenFE RMSE": np.mean(openfe_rmse),
    "OpenFE RMSE_lower": np.percentile(openfe_rmse, 2.5),
    "OpenFE RMSE_upper": np.percentile(openfe_rmse, 97.5),
    "FEP+ RMSE": np.mean(fep_plus_rmse),
    "FEP+ RMSE_lower": np.percentile(fep_plus_rmse, 2.5),
    "FEP+ RMSE_upper": np.percentile(fep_plus_rmse, 97.5)
}
# add the row to the dataframe using concat
dg_system_metrics = pd.concat([dg_system_metrics, pd.DataFrame([row_data])], ignore_index=True)
dg_system_metrics

In [ ]:
# paired significance test on the RMSE values not including the weighted RMSE
from scipy.stats import ttest_rel
# filter out the weighted RMSE row
dg_system_metrics_filtered = dg_system_metrics[dg_system_metrics["System"] != "Weighted\nRMSE"].copy(deep=True).reset_index(drop=True)
# perform the wilcoxon signed-rank test
from scipy.stats import wilcoxon
stat, p_value = wilcoxon(dg_system_metrics_filtered["OpenFE RMSE"], dg_system_metrics_filtered["FEP+ RMSE"])
print(f"Paired wilcoxon test between OpenFE and FEP+ RMSE: statistic = {stat:.3f}, p-value = {p_value}")

In [ ]:
# report the range of the RMSE values for OpenFE and FEP+ not including the weighted RMSE
openfe_rmse_range = dg_system_metrics_filtered["OpenFE RMSE"].agg(["min", "max"])
fep_plus_rmse_range = dg_system_metrics_filtered["FEP+ RMSE"].agg(["min", "max"])
print(f"OpenFE RMSE range: {openfe_rmse_range['min']:.3f} - {openfe_rmse_range['max']:.3f} kcal/mol")
print(f"FEP+ RMSE range: {fep_plus_rmse_range['min']:.3f} - {fep_plus_rmse_range['max']:.3f} kcal/mol")

In [ ]:
# plot the pairwise RMSE for OpenFE and FEP+ with 95% CI for each system and the weighted RMSE
x = np.arange(len(dg_system_metrics))

# Set up the figure
fig, ax = plt.subplots(figsize=(16, 8))
bar_width = 0.4

# Plot bars with error bars
rmse_err_openfe = [dg_system_metrics['OpenFE RMSE'] - dg_system_metrics['OpenFE RMSE_lower'], dg_system_metrics['OpenFE RMSE_upper'] - dg_system_metrics['OpenFE RMSE']]
rmse_err_plus = [dg_system_metrics['FEP+ RMSE'] - dg_system_metrics['FEP+ RMSE_lower'], dg_system_metrics['FEP+ RMSE_upper'] - dg_system_metrics['FEP+ RMSE']]
# "OpenFE": "#009384", "FEP+": "#d9c4b1"
ax.bar(x - bar_width/2, dg_system_metrics['OpenFE RMSE'], yerr=rmse_err_openfe, width=bar_width, label='OpenFE',
       color='#009384', capsize=3)
ax.bar(x + bar_width/2, dg_system_metrics['FEP+ RMSE'], yerr=rmse_err_plus, width=bar_width, label='FEP+',
       color="#d9c4b1", capsize=3)

# Set labels and ticks
ax.set_xticks(x)
names = dg_system_metrics['System'].str.replace("_", " ")
ax.set_xticklabels(names, rotation=90, fontsize=8)
ax.set_ylabel(r"Pairwise $\Delta\Delta$G$_{calc}$ RMSE (kcal/mol)")
# ax.set_title("Per-System RMSE and MUE with 95% Confidence Intervals")
ax.legend()

# Add colored background for classes
unique_classes = dg_system_metrics['Class'].unique()
# remove overall from the unique classes and add it to the end
unique_classes = [cls for cls in unique_classes if cls != "Overall"]
unique_classes.append("Overall")
class_bounds = dg_system_metrics.groupby('Class').size().cumsum().to_dict()
# move overall to the end
class_bounds["Overall"] = len(dg_system_metrics) + 1

for cls, bound in class_bounds.items():
    if cls == "Overall":
        continue
    class_bounds[cls] = bound - 1  # Adjust to match the last index of the class
print(class_bounds)
class_start = 0

colors = sns.color_palette("colorblind", len(unique_classes))
class_conversion = {"bayer_macrocycles": "Bayer\nMacrocycles", "charge_annihilation_set": "Charge\nAnnihilation", "fragments": "Fragments", "jacs_set": "JACS", "janssen_bace": "Janssen", "merck": "Merck", "miscellaneous_set": "Misc", "scaffold_hopping_set": "Scaffold\nHopping", "water_set": "Water", "mcs_docking_set": "MCS\nDocking"}
for i, cls in enumerate(unique_classes):
    cls_name = class_conversion.get(cls, cls)
    end = class_bounds[cls]
    # add dashed lines for the span but not the fill
    if cls != "charge_annihilation_set":
        ax.axvline(class_start - 0.5, linestyle='--', linewidth=2)
#     ax.axvspan(class_start - 0.5, end - 0.5, facecolor=colors[i], alpha=0.2)
    center = (class_start + end - 1) / 2
    ax.text(center, ax.get_ylim()[1] + 0.1, cls_name, ha='center', va='bottom', fontsize=10, weight='bold')
    class_start = end

plt.tight_layout()
plt.xlim(-0.5, len(dg_system_metrics) - 0.5)
plt.savefig("per_system_pairwise_rmse_openfe_vs_plus.png", dpi=300, bbox_inches='tight')

In [ ]:
# make a new dataframe with all possible pairwise DDG values for openfe and fep+ with symmetry
all_pairwise_ddgs = []
for system in all_dg_data["system group"].unique():
    # get the edges for these systems
    system_df = all_dg_data[all_dg_data["system group"] == system].copy(deep=True).reset_index(drop=True)
    targets = system_df["system name"].unique()
    for target in targets:
        # get the edges for this target
        target_df = system_df[(system_df["system name"] == target)].copy(deep=True).reset_index(drop=True)
        # get the ligands
        ligands = target_df["ligand name"].unique()
        for i, ligand1 in enumerate(ligands):
            for j, ligand2 in enumerate(ligands):
                if i >= j:  # skip self-comparisons
                    continue
                # get the ddg values for these ligands
                exp_dg1 = target_df[target_df["ligand name"] == ligand1]["Exp DG (kcal/mol)"].values[0]
                exp_dg2 = target_df[target_df["ligand name"] == ligand2]["Exp DG (kcal/mol)"].values[0]
                openfe_dg1 = target_df[target_df["ligand name"] == ligand1]["DG (kcal/mol)"].values[0]
                openfe_dg2 = target_df[target_df["ligand name"] == ligand2]["DG (kcal/mol)"].values[0]
                fep_plus_ddg1 = target_df[target_df["ligand name"] == ligand1]["FEP+ DG (kcal/mol)"].values[0]
                fep_plus_ddg2 = target_df[target_df["ligand name"] == ligand2]["FEP+ DG (kcal/mol)"].values[0]
                # add to a new dataframe
                new_row = {
                    "System": target,
                    "Class": system,
                    "Ligand 1": ligand1,
                    "Ligand 2": ligand2,
                    "Exp DDG (kcal/mol)": exp_dg2 - exp_dg1,
                    "OpenFE DDG (kcal/mol)": openfe_dg2 - openfe_dg1,
                    "FEP+ DDG (kcal/mol)": fep_plus_ddg2 - fep_plus_ddg1,
                }
                all_pairwise_ddgs.append(new_row)
all_pairwise_ddgs = pd.DataFrame(all_pairwise_ddgs)
all_pairwise_ddgs

In [ ]:
# plot ecdfs for openfe and fep+ ddg absolute errors
fig, ax = plt.subplots(figsize=(8, 6))
openfe_abs_ddg = np.abs(all_pairwise_ddgs["OpenFE DDG (kcal/mol)"] - all_pairwise_ddgs["Exp DDG (kcal/mol)"])
fep_plus_abs_ddg = np.abs(all_pairwise_ddgs["FEP+ DDG (kcal/mol)"] - all_pairwise_ddgs["Exp DDG (kcal/mol)"])
sns.ecdfplot(data=openfe_abs_ddg, ax=ax, label="OpenFE", color="#009384", linewidth=2)
sns.ecdfplot(data=fep_plus_abs_ddg, ax=ax, label="FEP+", color="#d9c4b1", linewidth=2)
# calculate the probability of being below 1 kcal/mol for both
prob_below_1_openfe = np.sum(openfe_abs_ddg < 1) / len(openfe_abs_ddg)
prob_below_1_fep_plus = np.sum(fep_plus_abs_ddg < 1) / len(fep_plus_abs_ddg)
print(f"Probability of OpenFE being below 1 kcal/mol: {prob_below_1_openfe:.2%}")
print(f"Probability of FEP+ being below 1 kcal/mol: {prob_below_1_fep_plus:.2%}")
# add a line at 1 kcal/mol for both
ax.axvline(x=1, ymax=prob_below_1_fep_plus, color='k', linestyle='--', linewidth=2)
ax.plot([0, 1], [prob_below_1_openfe, prob_below_1_openfe], color='#009384', linestyle='--', linewidth=2)
ax.plot([0, 1], [prob_below_1_fep_plus, prob_below_1_fep_plus], color='#d9c4b1', linestyle='--', linewidth=2)
# add a label above the line
ax.text(0.1, prob_below_1_openfe + 0.02, f"{prob_below_1_openfe:.2%}", color='#009384', fontsize=13)
ax.text(0.1, prob_below_1_fep_plus + 0.02, f"{prob_below_1_fep_plus:.2%}", color='#d9c4b1', fontsize=13)
# same again for 2  kcal/mol
prob_below_2_openfe = np.sum(openfe_abs_ddg < 2) / len(openfe_abs_ddg)
prob_below_2_fep_plus = np.sum(fep_plus_abs_ddg < 2) / len(fep_plus_abs_ddg)
print(f"Probability of OpenFE being below 2 kcal/mol: {prob_below_2_openfe:.0%}")
print(f"Probability of FEP+ being below 2 kcal/mol: {prob_below_2_fep_plus:.0%}")
# stop the line at the higher probability curve 

ax.axvline(x=2, ymax=prob_below_2_fep_plus, color='k', linestyle='--', linewidth=2)
ax.plot([0, 2], [prob_below_2_openfe, prob_below_2_openfe], color='#009384', linestyle='--', linewidth=2)
ax.plot([0, 2], [prob_below_2_fep_plus, prob_below_2_fep_plus], color='#d9c4b1', linestyle='--', linewidth=2)
# add a label above the line
ax.text(0.2, prob_below_2_openfe + 0.02, f"{prob_below_2_openfe:.2%}", color='#009384', fontsize=13)
ax.text(0.2, prob_below_2_fep_plus + 0.02, f"{prob_below_2_fep_plus:.2%}", color='#d9c4b1', fontsize=13)
# set the x and y labels
ax.set_xlabel(r"Pairwise $|\Delta\Delta$G$_{calc}-\Delta\Delta$G$_{exp}|$ (kcal/mol)", fontdict={"fontsize": 15})
ax.set_ylabel("Cumulative Probability", fontdict={"fontsize": 15})
# set x and y tick font size
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)
plt.xlim(left=0)
plt.legend(prop={"size": 15})
plt.tight_layout()
# save the figure
plt.savefig("ecdf_pairwise_abs_ddg_error_opnfe_vs_fep_plus.png", dpi=300, bbox_inches='tight')

In [ ]:
# bin the absolute experimental DDG values in 1 kcal/mol bins 
# calculate the probability of gettting the sign of the DDG correct with openfe and fep+ for each bin
def bin_sign_correctness(data, bin_size=1.0):
    # get the absolute experimental DDG values
    data['abs_exp_ddg'] = np.abs(data['Exp DDG (kcal/mol)'])
    # max value value to work out the bins
    max_value = data['abs_exp_ddg'].max()
    bins = np.arange(0, max_value + bin_size, bin_size)
    
    # Bin the data
    data['bin'] = pd.cut(data['abs_exp_ddg'], bins=bins, right=False)
    
    # Calculate probabilities
    results = []
    # sort the bins by the lower edge
    for b in sorted(data['bin'].unique(), key=lambda x: x.left):
        subset = data[data['bin'] == b]
        if len(subset) == 0:
            continue
        # Calculate the probability of correct sign for OpenFE and FEP+ and bootstrap the results
        openfe_correct = np.sum(np.sign(subset['OpenFE DDG (kcal/mol)']) == np.sign(subset['Exp DDG (kcal/mol)']))
        fep_plus_correct = np.sum(np.sign(subset['FEP+ DDG (kcal/mol)']) == np.sign(subset['Exp DDG (kcal/mol)']))
        total = len(subset)
        # Bootstrap the probabilities
        nboots = 1000
        openfe_probs = []
        fep_plus_probs = [] 
        for _ in range(nboots):
            bootstrapped_subset = subset.sample(n=len(subset), replace=True)
            openfe_correct_boot = np.sum(np.sign(bootstrapped_subset['OpenFE DDG (kcal/mol)']) == np.sign(bootstrapped_subset['Exp DDG (kcal/mol)']))
            fep_plus_correct_boot = np.sum(np.sign(bootstrapped_subset['FEP+ DDG (kcal/mol)']) == np.sign(bootstrapped_subset['Exp DDG (kcal/mol)']))
            total_boot = len(bootstrapped_subset)
            openfe_probs.append(openfe_correct_boot / total_boot)
            fep_plus_probs.append(fep_plus_correct_boot / total_boot)
        # Calculate mean and 95% CI
        openfe_mean = np.mean(openfe_probs)
        openfe_low = np.percentile(openfe_probs, 2.5)
        openfe_high = np.percentile(openfe_probs, 97.5)
        fep_plus_mean = np.mean(fep_plus_probs)
        fep_plus_low = np.percentile(fep_plus_probs, 2.5)
        fep_plus_high = np.percentile(fep_plus_probs, 97.5)
        # store the results so we can use hue to split the data
        results.append({
            'bin': b.left,
            'OpenFE Probability': openfe_mean,
            'OpenFE Probability Lower': openfe_low,
            'OpenFE Probability Upper': openfe_high,
            'FEP+ Probability': fep_plus_mean,
            'FEP+ Probability Lower': fep_plus_low,
            'FEP+ Probability Upper': fep_plus_high
        })
        # x+=1
    return pd.DataFrame(results)
# Calculate the binned probabilities
binned_probabilities = bin_sign_correctness(all_pairwise_ddgs, bin_size=0.5)

In [ ]:

# Plot the binned probabilities using a bar plot with error bars
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data=binned_probabilities, x='bin', y='OpenFE Probability', ax=ax, color='#009384', label='OpenFE', alpha=0.7, width=1.0)
sns.barplot(data=binned_probabilities, x='bin', y='FEP+ Probability', ax=ax, color='#d9c4b1', label='FEP+', alpha=0.7, width=1.0)
# add error bars for the probabilities
ax.set_xlabel(r"|$\Delta\Delta$G$_{exp}$| (kcal/mol)", fontsize=14)
ax.set_ylabel(r"Probability Correct pairwise $\Delta\Delta$G$_{calc}$ Sign", fontsize=14)
# set the x ticks to go from 0 to the max bin value in 1 kcal/mol increments
x_ticks = np.arange(1.5, 15.5, 2)
ax.errorbar(binned_probabilities['bin'] *2, binned_probabilities['OpenFE Probability'],
            yerr=[binned_probabilities['OpenFE Probability'] - binned_probabilities['OpenFE Probability Lower'],
                  binned_probabilities['OpenFE Probability Upper'] - binned_probabilities['OpenFE Probability']],
            fmt='none', color='black', capsize=5)
ax.errorbar(binned_probabilities['bin'] *2, binned_probabilities['FEP+ Probability'],
            yerr=[binned_probabilities['FEP+ Probability'] - binned_probabilities['FEP+ Probability Lower'],
                  binned_probabilities['FEP+ Probability Upper'] - binned_probabilities['FEP+ Probability']],
            fmt='none', color='black', capsize=5)
# add scatter points for the probabilities
ax.scatter(binned_probabilities['bin'] *2, binned_probabilities['OpenFE Probability'], color='#009384', edgecolor='black', s=50)
ax.scatter(binned_probabilities['bin'] *2, binned_probabilities['FEP+ Probability'], color='#d9c4b1', edgecolor='black', s=50)
# print(x_ticks)
# print(ax.get_xticks())
ax.set_xticks(x_ticks)
ax.set_xticklabels([1, 2, 3, 4, 5, 6, 7], fontsize=12)
plt.xlim((-0.5,14.5))
# save the figure
plt.legend(fontsize=12)
plt.tight_layout()
plt.ylim(bottom=0.4)
plt.savefig("binned_sign_correctness_openfe_vs_fep_plus.png", dpi=300, bbox_inches='tight')

In [ ]:
all_pairwise_ddgs

In [ ]:
# make a 12x5 grid of subplots with shared y axis and compare the openfe and fep absolute errors using ecdfs for each system name
fig, axs = plt.subplots(12, 5, figsize=(20, 40), sharey=True)
# Flatten the axes array for easier iteration
axs = axs.flatten()
i=0
# Loop through each system and plot the data  
system_groups = all_pairwise_ddgs["Class"].unique()
for system in system_groups:
    system_df = all_pairwise_ddgs[all_pairwise_ddgs["Class"] == system].copy(deep=True).reset_index(drop=True)
    # get the unique target names
    target_names = system_df["System"].unique()    
    for target in target_names:
        # get the edges for this target
        target_df = system_df[system_df["System"] == target].copy(deep=True).reset_index(drop=True)
        # calculate the absolute errors
        openfe_abs_ddg = np.abs(target_df["OpenFE DDG (kcal/mol)"] - target_df["Exp DDG (kcal/mol)"])
        fep_plus_abs_ddg = np.abs(target_df["FEP+ DDG (kcal/mol)"] - target_df["Exp DDG (kcal/mol)"])
        # plot the ecdf
        sns.ecdfplot(data=openfe_abs_ddg, ax=axs[i], label="OpenFE", color="#009384", linewidth=2)
        sns.ecdfplot(data=fep_plus_abs_ddg, ax=axs[i], label="FEP+", color="#d9c4b1", linewidth=2)
        axs[i].set_title(f"{system.replace('_', ' ').replace("set","")} - {target.replace('_', ' ')}", fontsize=10, fontweight='bold')
        i+= 1
# Set the y-label for all subplots
fig.supylabel("Cumulative Probability", fontsize=14)
# Set the x-label for all subplots
fig.supxlabel(r"Pairwise $|\Delta\Delta$G$_{calc}-\Delta\Delta$G$_{exp}|$ (kcal/mol)", fontsize=14)
# add the legend to the first subplot
axs[0].legend(prop={"size": 12})
# Adjust layout to prevent overlap
plt.tight_layout()
plt.savefig("ecdf_pairwise_abs_ddg_error_opnfe_vs_fep_plus_per_system.png", dpi=300, bbox_inches='tight')


In [ ]:
# get the overlapping edges between openfe and fep+
# we need to check for reverse edges as well
overlaping_edges = []
for _, row in all_openfe_edges.iterrows():
    data = row.to_dict()
    # find the row in the FEP+ data
    fep_group = fep_plus_edge_data[(fep_plus_edge_data["system group"] == row["system group"]) & (fep_plus_edge_data["system name"] == row["system name"])]
    # try the forward ligands
    fep_data_row = fep_group[(fep_group["Lig 1"] == row["ligand_A"]) & (fep_group["Lig 2"] == row["ligand_B"])]
    backwards = False
    if len(fep_data_row) == 0:
        # try the reverse
        fep_data_row = fep_group[(fep_group["Lig 2"] == row["ligand_A"]) & (fep_group["Lig 1"] == row["ligand_B"])]
        backwards = True
    if len(fep_data_row) == 1:
        factor = -1 if backwards else 1
        fep_data = fep_data_row.iloc[0]
        data["FEP+ Bennett ddG (kcal/mol)"] = fep_data["Bennett ddG (kcal/mol)"] * factor
        data["FEP+ Bennett std. error (kcal/mol)"] = fep_data["Bennett std. error (kcal/mol)"]
        data["FEP+ CCC ddG (kcal/mol)"] = fep_data["CCC ddG (kcal/mol)"] * factor
        data["FEP+ CCC std. error (kcal/mol)"] = fep_data["CCC std. error (kcal/mol)"]
        assert round(row["exp DDG (kcal/mol)"], ndigits=4)  == round(factor * fep_data["Exp. ddG (kcal/mol)"], ndigits=4), print(round(factor * fep_data["Exp. ddG (kcal/mol)"], ndigits=4), round(row["exp DDG (kcal/mol)"], ndigits=4))
        overlaping_edges.append(data)
    # else:
    #     print(f"can not find edge: {row['ligand_A']}-{row['ligand_B']}, {row['system group']}:{row['system name']}")

overlap_df = pd.DataFrame(overlaping_edges)
overlap_df



In [ ]:
# workout the % of edges which overlap per dataset
target_edge_overlap = []
for system in all_openfe_edges["system group"].unique():
    system_df = all_openfe_edges[all_openfe_edges["system group"] == system]
    targets = system_df["system name"].unique()
    for target in targets:
        target_df = system_df[system_df["system name"] == target]
        # get the targetdf from fep+
        fep_plus_target_df = fep_plus_edge_data[(fep_plus_edge_data["system group"] == system) & (fep_plus_edge_data["system name"] == target)]
        # loop over the edges
        total_openfe_edges = len(target_df)
        overlap_edges = 0
        for _, row in target_df.iterrows():
            # try and find this edge in the fep+ data
            fep_data_row = fep_plus_target_df[(fep_plus_target_df["Lig 1"] == row["ligand_A"]) & (fep_plus_target_df["Lig 2"] == row["ligand_B"])]
            if len(fep_data_row) == 0:
                # try the reverse
                fep_data_row = fep_plus_target_df[(fep_plus_target_df["Lig 2"] == row["ligand_A"]) & (fep_plus_target_df["Lig 1"] == row["ligand_B"])]
            if len(fep_data_row) == 1:
                overlap_edges += 1
        target_edge_overlap.append(
            {
                "system group": system,
                "system name": target,
                "Overlap %": (overlap_edges / total_openfe_edges) * 100,
                "OpenFE Edges": total_openfe_edges,
                "Overlap edges": overlap_edges
            }
        )
target_overlap_df = pd.DataFrame(target_edge_overlap)
target_overlap_df


In [ ]:
target_overlap_df.sort_values("Overlap %")

In [ ]:
# get only the overlapping edges with EXP data, removing the intermediates
overlap_df_final = overlap_df[overlap_df["exp DDG (kcal/mol)"].notna()].copy(deep=True).reset_index(drop=True)
# drop the one record with the missing bennett data
overlap_df_final = overlap_df_final[overlap_df_final["FEP+ Bennett ddG (kcal/mol)"].notna()].copy(deep=True).reset_index(drop=True)
overlap_df_final

In [ ]:
# plot the cdf of the absolute errors
# make the dataframe
fig, ax = plt.subplots(figsize=(8, 6))
complex_data = overlap_df_final[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
complex_error = complex_data.std(axis=1)
solvent_data = overlap_df_final[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
solvent_error = solvent_data.std(axis=1)
openfe_ddg = complex_dg - solvent_dg
openfe_abs_ddg = np.abs(openfe_ddg - overlap_df_final["exp DDG (kcal/mol)"])
fep_plus_ddg = overlap_df_final["FEP+ Bennett ddG (kcal/mol)"]
fep_plus_abs_ddg = np.abs(fep_plus_ddg - overlap_df_final["exp DDG (kcal/mol)"])
sns.ecdfplot(x=openfe_abs_ddg, label="OpenFE", color="#009384", linewidth=2, ax=ax)
sns.ecdfplot(x=fep_plus_abs_ddg, label="FEP+", color="#d9c4b1", linewidth=2, ax=ax)
# calculate the probability of being below 1 kcal/mol for both
prob_below_1_openfe = np.sum(openfe_abs_ddg < 1) / len(openfe_abs_ddg)
prob_below_1_fep_plus = np.sum(fep_plus_abs_ddg < 1) / len(fep_plus_abs_ddg)
print(f"Probability of OpenFE being below 1 kcal/mol: {prob_below_1_openfe:.2%}")
print(f"Probability of FEP+ being below 1 kcal/mol: {prob_below_1_fep_plus:.2%}")
# add a line at 1 kcal/mol for both
ax.axvline(x=1, ymax=prob_below_1_fep_plus, color='k', linestyle='--', linewidth=2)
ax.plot([0, 1], [prob_below_1_openfe, prob_below_1_openfe], color='#009384', linestyle='--', linewidth=2)
ax.plot([0, 1], [prob_below_1_fep_plus, prob_below_1_fep_plus], color='#d9c4b1', linestyle='--', linewidth=2)
# add a label above the line
ax.text(0.1, prob_below_1_openfe + 0.02, f"{prob_below_1_openfe:.2%}", color='#009384', fontsize=13)
ax.text(0.1, prob_below_1_fep_plus + 0.02, f"{prob_below_1_fep_plus:.2%}", color='#d9c4b1', fontsize=13)
# same again for 2  kcal/mol
prob_below_2_openfe = np.sum(openfe_abs_ddg < 2) / len(openfe_abs_ddg)
prob_below_2_fep_plus = np.sum(fep_plus_abs_ddg < 2) / len(fep_plus_abs_ddg)
print(f"Probability of OpenFE being below 2 kcal/mol: {prob_below_2_openfe:.0%}")
print(f"Probability of FEP+ being below 2 kcal/mol: {prob_below_2_fep_plus:.0%}")
# stop the line at the higher probability curve 

ax.axvline(x=2, ymax=prob_below_2_fep_plus, color='k', linestyle='--', linewidth=2)
ax.plot([0, 2], [prob_below_2_openfe, prob_below_2_openfe], color='#009384', linestyle='--', linewidth=2)
ax.plot([0, 2], [prob_below_2_fep_plus, prob_below_2_fep_plus], color='#d9c4b1', linestyle='--', linewidth=2)
# add a label above the line
ax.text(0.2, prob_below_2_openfe - 0.06, f"{prob_below_2_openfe:.2%}", color='#009384', fontsize=13)
ax.text(0.2, prob_below_2_fep_plus + 0.02, f"{prob_below_2_fep_plus:.2%}", color='#d9c4b1', fontsize=13)

plt.legend(prop={"size": 15})
plt.xlim(left=0.0)
plt.ylabel("Cumulative probability", fontdict={"size": 15})
plt.tick_params(labelsize=15)
plt.xlabel(r"|$\Delta\Delta$G$_{calc}$ - $\Delta\Delta$G$_{exp}$| (kcal/mol)", fontdict={"size": 15})
plt.savefig("edgewise_abs_ddg_eror_ecdf_vs_fep.png", bbox_inches="tight", dpi=300)

In [ ]:
# plot an ecdf of the difference between the openfe and fep+ ddg values
fig, ax = plt.subplots(figsize=(8, 6))
complex_data = overlap_df_final[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
complex_error = complex_data.std(axis=1)
solvent_data = overlap_df_final[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
solvent_error = solvent_data.std(axis=1)
openfe_ddg = complex_dg - solvent_dg
fep_plus_ddg = overlap_df_final["FEP+ Bennett ddG (kcal/mol)"]
ddg_diff = np.abs(openfe_ddg - fep_plus_ddg)
sns.ecdfplot(x=ddg_diff, linewidth=2, ax=ax)
# set y axis title
ax.set_ylabel("Cumulative Probability", fontdict={"size": 15})
# set x axis title
ax.set_xlabel(r"|$\Delta\Delta$G$_{OpenFE}$ - $\Delta\Delta$G$_{FEP+}$| (kcal/mol)", fontdict={"size": 15})
# work out the probability of being below 1 kcal/mol
prob_below_1 = np.sum(ddg_diff < 1) / len(ddg_diff)
print(f"Probability of OpenFE - FEP+ being below 1 kcal/mol: {prob_below_1:.2%}")
# add a line at 1 kcal/mol
ax.axvline(x=1, ymax=prob_below_1, color='k', linestyle='--', linewidth=2)
ax.plot([0, 1], [prob_below_1, prob_below_1], color='k', linestyle='--', linewidth=2)
# add a label above the line
ax.text(0.1, prob_below_1 + 0.02, f"{prob_below_1:.2%}", color='k', fontsize=13)
# work out the probability of being below 2 kcal/mol
prob_below_2 = np.sum(ddg_diff < 2) / len(ddg_diff)
print(f"Probability of OpenFE - FEP+ being below 2 kcal/mol: {prob_below_2:.0%}")
# add a line at 2 kcal/mol
ax.axvline(x=2, ymax=prob_below_2, color='k', linestyle='--', linewidth=2)
ax.plot([0, 2], [prob_below_2, prob_below_2], color='k', linestyle='--', linewidth=2)
# add a label above the line
ax.text(0.2, prob_below_2 + 0.02, f"{prob_below_2:.2%}", color='k', fontsize=13)
# workout the probability of being below 0.5 kcal/mol
prob_below_05 = np.sum(ddg_diff < 0.5) / len(ddg_diff)
print(f"Probability of OpenFE - FEP+ being below 0.5 kcal/mol: {prob_below_05:.2%}")
# add a line at 0.5 kcal/mol    
ax.axvline(x=0.5, ymax=prob_below_05, color='k', linestyle='--', linewidth=2)
ax.plot([0, 0.5], [prob_below_05, prob_below_05], color='k', linestyle='--', linewidth=2)
# add a label above the line
ax.text(0.05, prob_below_05 + 0.02, f"{prob_below_05:.2%}", color='k', fontsize=13)
# set the x and y tick font size
ax.tick_params(axis='x', labelsize=15)
ax.tick_params(axis='y', labelsize=15)
plt.xlim(left=0.0)
# plt.legend(prop={"size": 15})
plt.tight_layout()
plt.savefig("edgewise_ddg_diff_ecdf.png", bbox_inches="tight", dpi=300)

In [ ]:
# calculate the stats for the overlap edges doing the bootstrapping manually
# calculate the ddg values for openfe
complex_data = overlap_df_final[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
solvent_data = overlap_df_final[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
openfe_ddg = (complex_dg - solvent_dg).values
fep_plus_ddg = overlap_df_final["FEP+ Bennett ddG (kcal/mol)"].values
exp_ddg = overlap_df_final["exp DDG (kcal/mol)"].values
# calculate the RMSE and MUE for OpenFE and FEP+
def calculate_rmse_mue(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_pred - y_true) ** 2))
    mue = np.mean(np.abs(y_pred - y_true))
    return rmse, mue
openfe_rmse, openfe_mue, fep_plus_rmse, fep_plus_mue = np.zeros(1000), np.zeros(1000), np.zeros(1000), np.zeros(1000)
for i in range(1000):
    # resample the data
    indices = np.random.choice(len(openfe_ddg), size=len(openfe_ddg), replace=True)
    openfe_sample = openfe_ddg[indices]
    exp_sample = exp_ddg[indices]
    fep_plus_sample = fep_plus_ddg[indices]
    # calculate the RMSE and MUE
    openfe_rmse[i], openfe_mue[i] = calculate_rmse_mue(exp_sample, openfe_sample)
    fep_plus_rmse[i], fep_plus_mue[i] = calculate_rmse_mue(exp_sample, fep_plus_sample)
# calculate the mean and 95% CI for the RMSE and MUE
openfe_rmse_mean = np.mean(openfe_rmse)
openfe_rmse_low = np.percentile(openfe_rmse, 2.5)
openfe_rmse_high = np.percentile(openfe_rmse, 97.5)
openfe_mue_mean = np.mean(openfe_mue)   
openfe_mue_low = np.percentile(openfe_mue, 2.5)
openfe_mue_high = np.percentile(openfe_mue, 97.5)
fep_plus_rmse_mean = np.mean(fep_plus_rmse)
fep_plus_rmse_low = np.percentile(fep_plus_rmse, 2.5)
fep_plus_rmse_high = np.percentile(fep_plus_rmse, 97.5)
fep_plus_mue_mean = np.mean(fep_plus_mue)
fep_plus_mue_low = np.percentile(fep_plus_mue, 2.5)
fep_plus_mue_high = np.percentile(fep_plus_mue, 97.5)
# create a dataframe with the results
stats_df = pd.DataFrame({
    "Protocol": ["OpenFE", "OpenFE", "FEP+", "FEP+"],
    "Stat": ["RMSE", "MUE", "RMSE", "MUE"],
    "Mean": [openfe_rmse_mean, openfe_mue_mean, fep_plus_rmse_mean, fep_plus_mue_mean],
    "Low": [openfe_rmse_low, openfe_mue_low, fep_plus_rmse_low, fep_plus_mue_low],
    "High": [openfe_rmse_high, openfe_mue_high, fep_plus_rmse_high, fep_plus_mue_high]
})
# print the stats dataframe
print(stats_df)

In [ ]:
from scipy.stats import wilcoxon
stat, p = wilcoxon(openfe_ddg, fep_plus_ddg)
# any significant difference?
if p < 0.05:
    print("There is a significant difference between OpenFE and FEP+ DDG values.")
else:
    print("There is no significant difference between OpenFE and FEP+ DDG values.")
print(f"Wilcoxon signed-rank test statistic: {stat}, p-value: {p}")

In [ ]:
# how many edges are charge changes
overlap_df_final[overlap_df_final["alchemical_charge_difference"] != 0.0]

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")
# sns.set_theme(style="ticks")
charge_change_data = []
charged_edges = overlap_df_final[overlap_df_final["alchemical_charge_difference"] != 0.0].copy(deep=True).reset_index(drop=True)
complex_data = charged_edges[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
solvent_data = charged_edges[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
openfe_ddg = complex_dg - solvent_dg
charged_edges["abs_error"] = np.abs(openfe_ddg - charged_edges["exp DDG (kcal/mol)"])
fep_plus_error = np.abs(charged_edges["FEP+ Bennett ddG (kcal/mol)"] - charged_edges["exp DDG (kcal/mol)"])

# wilcoxon test for the predictions
stat, p = wilcoxon(openfe_ddg, charged_edges["FEP+ Bennett ddG (kcal/mol)"])
print(f"Wilcoxon signed-rank test statistic for absolute errors: {stat}, p-value: {p}")
if p < 0.05:
    print("There is a significant difference between OpenFE and FEP+ absolute errors for charge change edges.")
# plot the absolute errors for OpenFE and FEP+ with a boxplot and stripplot
fig, ax = plt.subplots(figsize=(5, 6))
sns.boxplot(y=charged_edges["abs_error"], ax=ax, x=0, color="#009384", boxprops={"alpha": 0.4}, fliersize=0, width=0.6)
sns.boxplot(y=fep_plus_error, ax=ax, x=0.5, color="#d9c4b1", boxprops={"alpha": 0.4}, fliersize=0, width=0.6)
# add the points to the boxplot
sns.stripplot(y=charged_edges["abs_error"], x=0, ax=ax, color="#009384", size=5)
sns.stripplot(y=fep_plus_error, x=0.5, ax=ax, color="#d9c4b1", size=5)
# ax.set_xticks([0, 0.5])
ax.set_xticklabels(["OpenFE", "FEP+"], fontsize=15)
ax.set_ylabel(r"$|\Delta\Delta$G$_{calc} - \Delta\Delta$G$_{exp}|$ (kcal/mol)", fontsize=15)
# ax.set_xlabel("Charge difference", fontsize=15)
ax.tick_params(labelsize=15)
plt.tight_layout()
sns.despine(offset=10)
plt.savefig("charge_change_abs_error_boxplot.png", dpi=300, bbox_inches='tight')



In [ ]:
# calculate the RMSE and MUE for the charged edges and the neutral edges with bootstrapping
charged_edges = overlap_df_final[overlap_df_final["alchemical_charge_difference"] != 0.0].copy(deep=True).reset_index(drop=True)
neutral_edges = overlap_df_final[overlap_df_final["alchemical_charge_difference"] == 0.0].copy(deep=True).reset_index(drop=True)
# calculate the ddg values for openfe
complex_data = charged_edges[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
solvent_data = charged_edges[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
openfe_charged_ddg = (complex_dg - solvent_dg).values
fep_plus_charged_ddg = charged_edges["FEP+ Bennett ddG (kcal/mol)"].values
exp_charged_ddg = charged_edges["exp DDG (kcal/mol)"].values
# calculate the ddg values for openfe
complex_data = neutral_edges[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
solvent_data = neutral_edges[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
openfe_neutral_ddg = (complex_dg - solvent_dg).values
fep_plus_neutral_ddg = neutral_edges["FEP+ Bennett ddG (kcal/mol)"].values
exp_neutral_ddg = neutral_edges["exp DDG (kcal/mol)"].values
# calculate the RMSE and MUE for OpenFE and FEP+ for charged edges
charged_openfe_rmse, charged_fep_plus_rmse = np.zeros(1000), np.zeros(1000)
neutral_openfe_rmse, neutral_fep_plus_rmse = np.zeros(1000), np.zeros(1000)
for i in range(1000):
    # resample the charged edges
    indices = np.random.choice(len(openfe_charged_ddg), size=len(openfe_charged_ddg), replace=True)
    charged_openfe_sample = openfe_charged_ddg[indices]
    charged_exp_sample = exp_charged_ddg[indices]
    charged_fep_plus_sample = fep_plus_charged_ddg[indices]
    # calculate the RMSE using the function from above
    charged_openfe_rmse[i], _ = calculate_rmse_mue(charged_exp_sample, charged_openfe_sample)
    charged_fep_plus_rmse[i], _ = calculate_rmse_mue(charged_exp_sample, charged_fep_plus_sample)
    
    # resample the neutral edges
    indices = np.random.choice(len(openfe_neutral_ddg), size=len(openfe_neutral_ddg), replace=True)
    neutral_openfe_sample = openfe_neutral_ddg[indices]
    neutral_exp_sample = exp_neutral_ddg[indices]
    neutral_fep_plus_sample = fep_plus_neutral_ddg[indices]
    # calculate the RMSE and MUE
    neutral_openfe_rmse[i], _ = calculate_rmse_mue(neutral_exp_sample, neutral_openfe_sample)
    neutral_fep_plus_rmse[i], _ = calculate_rmse_mue(neutral_exp_sample, neutral_fep_plus_sample)

# calculate the mean and 95% CI for the RMSE and MUE
charged_openfe_rmse_mean = np.mean(charged_openfe_rmse)
charged_openfe_rmse_low = np.percentile(charged_openfe_rmse, 2.5)
charged_openfe_rmse_high = np.percentile(charged_openfe_rmse, 97.5)
charged_fep_plus_rmse_mean = np.mean(charged_fep_plus_rmse)
charged_fep_plus_rmse_low = np.percentile(charged_fep_plus_rmse, 2.5)
charged_fep_plus_rmse_high = np.percentile(charged_fep_plus_rmse, 97.5)
neutral_openfe_rmse_mean = np.mean(neutral_openfe_rmse)
neutral_openfe_rmse_low = np.percentile(neutral_openfe_rmse, 2.5)
neutral_openfe_rmse_high = np.percentile(neutral_openfe_rmse, 97.5)
neutral_fep_plus_rmse_mean = np.mean(neutral_fep_plus_rmse)
neutral_fep_plus_rmse_low = np.percentile(neutral_fep_plus_rmse, 2.5)
neutral_fep_plus_rmse_high = np.percentile(neutral_fep_plus_rmse, 97.5)
# create a dataframe with the results
stats_charge_df = pd.DataFrame({
    "Protocol": ["OpenFE", "OpenFE", "FEP+", "FEP+"],
    "Charge": ["Charged", "Neutral", "Charged", "Neutral"],
    "RMSE Mean": [charged_openfe_rmse_mean, neutral_openfe_rmse_mean, charged_fep_plus_rmse_mean, neutral_fep_plus_rmse_mean],
    "RMSE Low": [charged_openfe_rmse_low, neutral_openfe_rmse_low, charged_fep_plus_rmse_low, neutral_fep_plus_rmse_low],
    "RMSE High": [charged_openfe_rmse_high, neutral_openfe_rmse_high, charged_fep_plus_rmse_high, neutral_fep_plus_rmse_high]
})
# print the stats dataframe
print(stats_charge_df)

In [ ]:
# sort the edges by the absolute error of the openfe ddg values
overlap_df_final["abs_error"] = np.abs(overlap_df_final["exp DDG (kcal/mol)"] - openfe_ddg)
overlap_df_final = overlap_df_final.sort_values("abs_error", ascending=False).reset_index(drop=True)
overlap_df_final.head(10)

In [ ]:
all_dg_data

In [ ]:
# create a 12 by 5 grid of subplots for the openfe and FEP+ dg values compared to the experimental values include error bars RMSE, MUE R2 and kendall tau should be in the legend
fig, axs = plt.subplots(12, 5, figsize=(20, 40))
# Flatten the axes array for easier iteration
axs = axs.flatten()
i = 0
# Loop through each system and plot the data
system_groups = all_dg_data["system group"].unique()
for system in system_groups:
    system_df = all_dg_data[all_dg_data["system group"] == system].copy(deep=True).reset_index(drop=True)
    # get the unique target names
    target_names = system_df["system name"].unique()
    for target in target_names:
        # get the edges for this target
        target_df = system_df[system_df["system name"] == target].copy(deep=True).reset_index(drop=True)
        # plot the data
        sns.scatterplot(x=target_df["Exp DG (kcal/mol)"], y=target_df["DG (kcal/mol)"], ax=axs[i], color="#009384", label="OpenFE", s=50, edgecolor='black')
        sns.scatterplot(x=target_df["Exp DG (kcal/mol)"], y=target_df["FEP+ DG (kcal/mol)"], ax=axs[i], color="#d9c4b1", label="FEP+", s=50, edgecolor='black')
        # add the error bars for 
        axs[i].errorbar(target_df["Exp DG (kcal/mol)"], target_df["DG (kcal/mol)"], 
                        yerr=target_df["uncertainty (kcal/mol)"], fmt='none', color="#009384", capsize=5, elinewidth=1) 
        axs[i].errorbar(target_df["Exp DG (kcal/mol)"], target_df["FEP+ DG (kcal/mol)"], 
                        yerr=target_df["FEP+ DG std. error (kcal/mol)"], fmt='none', color="#d9c4b1", capsize=5, elinewidth=1)
        # calculate the RMSE and MUE for OpenFE and FEP+
        openfe_rmse, openfe_mue = calculate_rmse_mue(target_df["Exp DG (kcal/mol)"].values, target_df["DG (kcal/mol)"].values)
        fep_plus_rmse, fep_plus_mue = calculate_rmse_mue(target_df["Exp DG (kcal/mol)"].values, target_df["FEP+ DG (kcal/mol)"].values)
        # calculate the R2 and Kendall tau
        axs[i].set_title(f"{system.replace('_', ' ').replace("set","")} - {target.replace('_', ' ')}", fontsize=10, fontweight='bold')

        i += 1
axs[0].legend(title="Protocol", fontsize=10, loc='upper left')
plt.tight_layout() 

In [ ]:
from scipy.stats import kendalltau
def calculate_kendall_tau(df, system_group, system_name):
    target_df = df[(df["system group"] == system_group) & (df["system name"] == system_name)].copy(deep=True).reset_index(drop=True)
    openfe_tau, _ = kendalltau(target_df["Exp DG (kcal/mol)"], target_df["DG (kcal/mol)"])
    fep_plus_tau, _ = kendalltau(target_df["Exp DG (kcal/mol)"], target_df["FEP+ DG (kcal/mol)"])
    return openfe_tau, fep_plus_tau


In [ ]:
calculate_kendall_tau(all_dg_data, "fragments", "liga")

In [ ]:
# workout the errors for the overlapping dataset!
# get the openfe error
exp_ddg = overlap_df_final["exp DDG (kcal/mol)"]
# exp_error = df["exp dDDG (kcal/mol)"]
# calculate the DDG using the std between repeats as the error
complex_data = overlap_df_final[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
complex_error = complex_data.std(axis=1)
solvent_data = overlap_df_final[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
solvent_error = solvent_data.std(axis=1)
ddg = complex_dg - solvent_dg
openfe_error = abs(ddg - exp_ddg)
fep_error = abs(exp_ddg - overlap_df_final["FEP+ Bennett ddG (kcal/mol)"])
overlap_high_error = overlap_df_final[(fep_error > 1.5) & (openfe_error > 1.5)].copy(deep=True).reset_index(drop=True)
# overlap_high_error
overlap_openfe_better = overlap_df_final[openfe_error + 1.0 < fep_error].copy(deep=True).reset_index(drop=True)
overlap_fep_plus_better = overlap_df_final[fep_error + 1.0 < openfe_error]


In [ ]:
overlap_openfe_better

In [ ]:
# for the openfe better edges print the exp ddg, openfe ddg, fep+ ddg and if openfe and FEP+ have the same sign
openfe_sign_correct = 0
fep_plus_sign_correct = 0
for _, row in overlap_openfe_better.iterrows():
    exp_ddg = row["exp DDG (kcal/mol)"]
    complex_data = row[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
    complex_dg = complex_data.mean()
    solvent_data = row[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
    solvent_dg = solvent_data.mean()
    openfe_ddg = complex_dg - solvent_dg
    fep_plus_ddg = row["FEP+ Bennett ddG (kcal/mol)"]
    # get the % of correct sign for openfe and fep+
    if np.sign(openfe_ddg) == np.sign(exp_ddg):
        openfe_sign_correct += 1
    if np.sign(fep_plus_ddg) == np.sign(exp_ddg):
        fep_plus_sign_correct += 1
print(f"OpenFE sign correct: {openfe_sign_correct} / {len(overlap_openfe_better)}")
print(f"FEP+ sign correct: {fep_plus_sign_correct} / {len(overlap_openfe_better)}")

In [ ]:
overlap_fep_plus_better

In [ ]:
openfe_sign_correct = 0
fep_plus_sign_correct = 0
for _, row in overlap_fep_plus_better.iterrows():
    exp_ddg = row["exp DDG (kcal/mol)"]
    complex_data = row[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
    complex_dg = complex_data.mean()
    solvent_data = row[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
    solvent_dg = solvent_data.mean()
    openfe_ddg = complex_dg - solvent_dg
    fep_plus_ddg = row["FEP+ Bennett ddG (kcal/mol)"]
    # get the % of correct sign for openfe and fep+
    if np.sign(openfe_ddg) == np.sign(exp_ddg):
        openfe_sign_correct += 1
    if np.sign(fep_plus_ddg) == np.sign(exp_ddg):
        fep_plus_sign_correct += 1
print(f"OpenFE sign correct: {openfe_sign_correct} / {len(overlap_fep_plus_better)}")
print(f"FEP+ sign correct: {fep_plus_sign_correct} / {len(overlap_fep_plus_better)}")

In [ ]:
overlap_high_error.to_csv("overlap_high_error_edges.csv")
overlap_openfe_better.to_csv("overlap_openfe_better.csv")
overlap_fep_plus_better.to_csv("overlap_fep_plus_better.csv")



In [ ]:
# box plot of differences find the list of outliers
diff_prediction = abs(ddg - overlap_df_final["FEP+ Bennett ddG (kcal/mol)"])
sns.boxplot(diff_prediction)


In [ ]:
# find systems with a lot of large differences
complex_data = overlap_df_final[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
solvent_data = overlap_df_final[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
ddg = complex_dg - solvent_dg
overlap_df_final["ABS diff predition"] = abs(ddg - overlap_df_final["FEP+ Bennett ddG (kcal/mol)"])
sorted_by_abs_diff = overlap_df_final.sort_values("ABS diff predition").copy(deep=True).reset_index(drop=True)

In [ ]:
sorted_by_abs_diff[sorted_by_abs_diff["ABS diff predition"] < 0.5]

In [ ]:
system_counts = {}
for _, row in sorted_by_abs_diff[sorted_by_abs_diff["ABS diff predition"] > 0.5].iterrows():
    name = f"{row['system group']}/{row['system name']}"
    if name not in system_counts:
        system_counts[name] = 1
    else:
        system_counts[name] += 1
system_counts

In [ ]:
# sort the systems by the number of edges with a large difference
sorted_systems = sorted(system_counts.items(), key=lambda x: x[1], reverse=True)
for system, count in sorted_systems:
    print(f"{system}: {count} edges with a large difference")

In [ ]:
# load all the ligands
import pathlib
from rdkit import Chem
all_ligands = {}
name_conversions = {
    "41 flip": "41-flip",
    "40 flip": "40-flip",
    "38 flip": "38-flip",
    "30 flip": "30-flip",
    "43 flip": "43-flip",
    "47 flip": "47-flip",
    "48 flip": "48-flip",
    "46 flip": "46-flip",
    "36 out": "36o",
    "37 out": "37o",
    "38 out": "38o",
    "39 out": "39o",
    "28 out": "28o",
    "CHEMBL3402756_2.7 redocked": "CHEMBL3402756_2.7_redocked",
    "CHEMBL3402757_6.5 redocked" : "CHEMBL3402757_6.5_redocked",
    "CHEMBL3402758_10 redocked": "CHEMBL3402758_10_redocked",
    "CHEMBL3402760_1 redocked":"CHEMBL3402760_1_redocked",
    "CHEMBL3402762_1 redocked": "CHEMBL3402762_1_redocked",
    "CHEMBL3402759_5.7 redocked": "CHEMBL3402759_5.7_redocked",
    "CHEMBL3402761_1 redocked": "CHEMBL3402761_1_redocked",
    "Example 22":"Example-22",
    "Example 23": "Example-23",
    "Example 14": "Example-14",
    "Example 9": "Example-9",
    "SHP099-1 Example 7": "SHP099-1-Example-7",
    "Example 28": "Example-28",
    "Example 24": "Example-24",
    "Example 26": "Example-26",
    "Example 6": "Example-6",
    "Example 1": "Example-1",
    "Example 30": "Example-30",
    "Example 8": "Example-8",
    "Example 29": "Example-29",
    "Example 2": "Example-2",
    "Example 25": "Example-25",
    "Example 4": "Example-4",
    "Example 3": "Example-3",
    "Example 27": "Example-27",
    "Example 5": "Example-5",
    "9 flip": "9-flip",
}
key_to_ligand = {}
base_data_folder = pathlib.Path("/Users/joshua/Documents/Software/IndustryBenchmarks2024/industry_benchmarks/input_structures/prepared_structures")
for folder in base_data_folder.glob("*"):
    if folder.is_dir() and folder != "template":
        for target_ligs in folder.glob("*/ligands.sdf"):
            # load the ligands
            supplier = Chem.SDMolSupplier(target_ligs, removeHs=False)
            for lig in supplier:
                name = lig.GetProp("_Name")
                if name in name_conversions:
                    name = name_conversions[name]
                all_ligands[(name, target_ligs.parent.name, folder.name)] = Chem.GetFormalCharge(lig)
                key_to_ligand[(name, target_ligs.parent.name, folder.name)] = lig

In [ ]:
from gufe import SmallMoleculeComponent
import kartograf
from kartograf.filters import (
    filter_ringbreak_changes,
    filter_ringsize_changes,
    filter_whole_rings_only,
)
from rdkit.Chem import Draw
grid_x, grid_y = 2, 1
from gufe.visualization.mapping_visualization import draw_mapping

def mapping_karto(liga, ligb, target, system):
    d2d = Draw.rdMolDraw2D.MolDraw2DSVG(grid_x * 300, grid_y * 300, 300, 300)
    mapping_filters = [
            filter_ringbreak_changes,  # default
            filter_ringsize_changes,  # default
            filter_whole_rings_only,  # default
        ]
    mapper = kartograf.KartografAtomMapper(
        # atom_max_distance=10,
        atom_map_hydrogens=True,
        additional_mapping_filter_functions=mapping_filters,
        # map_hydrogens_on_hydrogens_only=True
    )
    ligand_a = SmallMoleculeComponent(key_to_ligand[(liga, target, system)])
    ligand_b = SmallMoleculeComponent(key_to_ligand[(ligb, target, system)])
    mapping =  next(mapper.suggest_mappings(ligand_a, ligand_b))
    return draw_mapping(mapping._compA_to_compB, mapping.componentA.to_rdkit(), mapping.componentB.to_rdkit(), d2d)

In [ ]:
overlap_high_error

In [ ]:
# function to automatically generate a pandas html report of the overlapping sets
# get the atom mapping exp openfe and FEP values and plot a bar chart to quickly look at the data and write to HTML
import base64
import numpy as np
def mol_to_html(interaction_image: str) -> str:
    raw_data = base64.b64encode(interaction_image.encode()).decode()
    return f"<img src='data:image/svg+xml;base64,{raw_data}'/>"

def create_overlap_report(df, fname):
    report_data = []
    for _, row in df.iterrows():
        system_name = row["system name"]
        system_group = row["system group"]
        liga = row["ligand_A"]
        ligb = row["ligand_B"]
        row_data = {"mapping": mapping_karto(liga=liga, ligb=ligb, target=system_name, system=system_group), "Exp DDG (kcal/mol)": row["exp DDG (kcal/mol)"]}
        # workout the openfe estimate
        complex_data = [row[f"complex_repeat_{i}_DG (kcal/mol)"] for i in range(3)]
        complex_dg = np.mean(complex_data)
        complex_error = np.std(complex_data, ddof=1)
        solvent_data = [row[f"solvent_repeat_{i}_DG (kcal/mol)"] for i in range(3)]
        solvent_dg = np.mean(solvent_data)
        solvent_error = np.std(solvent_data, ddof=1)
        row_data["OpenFE DDG"] = complex_dg - solvent_dg
        row_data["OpenFE dDDG"] = (complex_error ** 2 + solvent_error ** 2) ** 0.5
        row_data["FEP+ Bennett DDG"] = row["FEP+ Bennett ddG (kcal/mol)"]
        row_data["FEP+ Bennett dDDG"] = row["FEP+ Bennett std. error (kcal/mol)"]
        row_data["system group"] = system_group
        row_data["system name"] = system_name
        report_data.append(row_data)
    report = pd.DataFrame(report_data)
    report.to_html(fname, escape=False, formatters={"mapping": mol_to_html})
    

In [ ]:
# create a report for the overlap edges from jacs set p38
p38_overlap_edges = overlap_df_final[(overlap_df_final["system group"] == "jacs_set") & (overlap_df_final["system name"] == "p38")].copy(deep=True).reset_index(drop=True)
create_overlap_report(p38_overlap_edges, "p38_overlap_edges.html")

In [ ]:
create_overlap_report(overlap_fep_plus_better, "fep_plus_better_edges.html")

In [ ]:
create_overlap_report(overlap_openfe_better, "overlap_openfe_better_report.html")

In [ ]:
# load the bootstrap data
bootstrap_edges_openfe = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/processed_results/combined_pymbar4_edge_data.csv")
rerun_bootstrap_openfe = pd.read_csv("https://raw.githubusercontent.com/OpenFreeEnergy/IndustryBenchmarks2024/refs/heads/main/industry_benchmarks/analysis/processed_results/reruns/rerun_pymbar4_edge_data.csv")
all_bootstrap_edges_openfe = bootstrap_edges_openfe[bootstrap_edges_openfe["system name"] != "pfkfb3"].copy(deep=True).reset_index(drop=True)
all_bootstrap_edges_openfe = pd.concat([all_bootstrap_edges_openfe, rerun_bootstrap_openfe]).reset_index(drop=True)
all_bootstrap_edges_openfe

In [ ]:
all_openfe_edges

In [ ]:
# get all complex mbar errors and compare with the std between repeats and bootstrap errors
mbar_data = []
for system in all_openfe_edges["system group"].unique():
    system_df = all_openfe_edges[all_openfe_edges["system group"] == system].copy(deep=True).reset_index(drop=True)
    for target in system_df["system name"].unique():
        target_df = system_df[system_df["system name"] == target].copy(deep=True).reset_index(drop=True)
        bootstrap_target_df = all_bootstrap_edges_openfe[(all_bootstrap_edges_openfe["system group"] == system) & (all_bootstrap_edges_openfe["system name"] == target)]
        for _, row in target_df.iterrows():
            data = {
                "Standard error of repeates (kcal/mol)": np.std([row[f"complex_repeat_{i}_DG (kcal/mol)"] for i in range(3)], ddof=1),
                "Error type": "Analytical",
                "Error estimate (kcal/mol)": row["complex_repeat_0_dDG (kcal/mol)"]
            }
            # get the bootstrap data
            boot_row = bootstrap_target_df[(bootstrap_target_df["ligand_A"] == row["ligand_A"]) & (bootstrap_target_df["ligand_B"] == row["ligand_B"])]
            if len(boot_row) == 1:
                data_boot = {
                    "Standard error of repeates (kcal/mol)": np.std([row[f"complex_repeat_{i}_DG (kcal/mol)"] for i in range(3)], ddof=1),
                    "Error type": "Bootstrap",
                    "Error estimate (kcal/mol)": boot_row.iloc[0]["complex_repeat_0_dDG (kcal/mol)"]
                }
                mbar_data.extend([data, data_boot])

pd.DataFrame(mbar_data)

In [ ]:
pd.DataFrame(mbar_data).sort_values("Error estimate (kcal/mol)")

In [ ]:
sns.set_theme("paper")
sns.set_style("darkgrid")
sns.kdeplot(data=pd.DataFrame(mbar_data), x="Error estimate (kcal/mol)", y="Standard error of repeates (kcal/mol)", hue="Error type")
plt.xlabel("Error estimate (kcal/mol)", fontdict={"size": 15})
plt.ylabel("Standard error of repeats (kcal/mol)", fontdict={"size": 15})
plt.tick_params(labelsize=12)
plt.xlim((0, 2))
plt.ylim((0, 5))
# plt.legend(fontsize="x-large")
plt.savefig("paper_figures/mbar_error_comp_kde.pdf", bbox_inches="tight")

In [ ]:
# compre the overlap from mbar with the absolute edge error
exp_ddg = variable_edges["exp DDG (kcal/mol)"]
exp_error = variable_edges["exp dDDG (kcal/mol)"]
# calculate the DDG using the std between repeats as the error
complex_data = variable_edges[[f"complex_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
complex_dg = complex_data.mean(axis=1)
complex_error = complex_data.std(axis=1)
solvent_data = variable_edges[[f"solvent_repeat_{i}_DG (kcal/mol)" for i in range(3)]]
solvent_dg = solvent_data.mean(axis=1)
solvent_error = solvent_data.std(axis=1)
mean_ddg = complex_dg - solvent_dg
ddg_error = (complex_error ** 2 + solvent_error ** 2) ** 0.5
abs_mean_error = abs(mean_ddg - exp_ddg)
# repeat 0 
ddg_0 = variable_edges["complex_repeat_2_DG (kcal/mol)"] - variable_edges["solvent_repeat_2_DG (kcal/mol)"]
mean_0_abs_error =  abs(ddg_0 - exp_ddg)
